### 전체 지하철역 데이터 전처리(9호선은 없는듯 함)

In [4]:
import warnings
warnings.filterwarnings(action='ignore')
import scipy as sp
import scipy.stats
import statsmodels.api as sm
import sklearn as sk
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt
import glob
import pandas as pd
set(sorted([f.name for f in mpl.font_manager.fontManager.ttflist]))
mpl.rc('font', family='NanumGothic')
mpl.rc('axes', unicode_minus=False)

In [5]:
path ='C:/Code/regression'
filenames = glob.glob(path + "/*.csv")

In [6]:
sub_2015=pd.DataFrame(pd.read_csv(filenames[0],encoding='euc-kr',thousands = ','))
sub_2016=pd.DataFrame(pd.read_csv(filenames[1],encoding='euc-kr',thousands = ','))
sub_2017=pd.DataFrame(pd.read_csv(filenames[2],encoding='euc-kr',thousands = ','))
sub_2018=pd.DataFrame(pd.read_csv(filenames[3],encoding='euc-kr',thousands = ','))
sub_2019=pd.DataFrame(pd.read_csv(filenames[4],encoding='euc-kr',thousands = ','))

IndexError: list index out of range

In [7]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 6850-4FA4

 C:\dss13\회귀분석 프로젝트 디렉터리

2020-06-11  오후 08:16    <DIR>          .
2020-06-11  오후 08:16    <DIR>          ..
2020-06-11  오후 08:14    <DIR>          .ipynb_checkpoints
2020-06-11  오후 08:13        13,017,253 2015sub.csv
2020-06-11  오후 08:13        13,055,897 2016sub.csv
2020-06-11  오후 08:13        12,684,562 2017sub.csv
2020-06-11  오후 08:13        11,462,196 2018sub.csv
2020-06-11  오후 08:13        11,470,440 2019sub.csv
2020-06-11  오후 08:14             2,725 holi.csv
2020-06-11  오후 08:16            14,635 sub_data.ipynb
               7개 파일          61,707,708 바이트
               3개 디렉터리  13,686,947,840 바이트 남음


In [8]:
sub_2015 = pd.read_csv('2015sub.csv',encoding='euc-kr',thousands=',')
sub_2016 = pd.read_csv('2015sub.csv',encoding='euc-kr',thousands=',')
sub_2016 = pd.read_csv('2015sub.csv',encoding='euc-kr',thousands=',')
sub_2016 = pd.read_csv('2015sub.csv',encoding='euc-kr',thousands=',')
sub_2015 = pd.read_csv('2015sub.csv',encoding='euc-kr',thousands=',')


In [ ]:
sub_2015.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2016.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2017.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2018.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2019.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']

In [ ]:
sub=pd.concat([sub_2015,sub_2016,sub_2017,sub_2018,sub_2019])

In [ ]:
sub

In [ ]:
# 행 개수 확인
len(sub_2015)+len(sub_2016)+len(sub_2017)+len(sub_2018)+len(sub_2019)

In [ ]:
# index 재할당
sub.reset_index(inplace=True,drop=True)

In [ ]:
# to_numeric
sub[['역번호', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']]=sub[['역번호', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']].apply(pd.to_numeric)

In [ ]:
sub.날짜=pd.to_datetime(sub['날짜'])

In [ ]:
sub['year']=sub.날짜.apply(lambda x: x.year)
sub['month']=sub.날짜.apply(lambda x: x.month)

In [ ]:
# 요일 찾기
import datetime
from datetime import date

t=['월','화','수','목','금','토','일']
sub["요일"]=sub['날짜'].apply(lambda x: t[x.weekday()])

In [ ]:
sub

```




```

### 2호선 퇴근시간만

In [ ]:
# 2호선 퇴근시간만
sub2=sub[(200<sub["역번호"]) & (sub["역번호"]<300)].reset_index(drop=True)
sub2.drop(['7to8', '8to9', '9to10'],1,inplace=True)

In [ ]:
sub2

In [ ]:
# 17~20시간대의 탑승객
sub2["17to20"]=sub2["17to18"]+sub2["18to19"]+sub2["19to20"]

In [ ]:
# 역명에서 역번호 제외
sub2["역명"]=sub2.역명.str.split("(",expand=True)[0]

In [ ]:
# 왜 위에꺼는 [0]하면 나오는데 이건 안나옴?????????????????????
# 둘다 똑같이 dataframe인데..
sub2[0]

In [ ]:
sub2.drop(["역번호","17to18","18to19","19to20"],1,inplace=True)

In [ ]:
sub2

In [ ]:
len(sub2.역명.unique()),sub2.역명.unique()

In [ ]:
# 신천이 잠실새내로 변경되어 두개의 이름 존재
sub2.loc[sub2['역명']=="신천",'역명']="잠실새내"

In [ ]:
sub2

In [ ]:
len(sub2.역명.unique()),sub2.역명.unique()

In [ ]:
np.sum(sub2.isnull())

### 2호선 데이터 공휴일 여부 표시

In [ ]:
holi=pd.read_csv("holi.csv",encoding="euc-kr")

In [ ]:
holi

In [ ]:
# seq 공휴일이 겹친경우 카운트 됨
holi[holi.seq==2].locdate

In [ ]:
# 띠용 20171002는 seq 2 인데 한개 밖에 없눈데..
holi[(holi.locdate==20171002) | (holi.locdate==20171003)]

In [ ]:
# isHoliday가 N인경우는 쉬는날이 아니었으므로 제거
holi=holi[holi.isHoliday!="N"]

In [ ]:
holi.groupby("year").size()

In [ ]:
holi.locdate=pd.to_datetime(holi.locdate,format="%Y%m%d")

In [ ]:
# sub2.holi는 공휴일 -> 주말은 아님
sub2["holi"]=sub2["날짜"].isin(holi.locdate)

In [ ]:
sub2[sub2["year"]==2017][sub2.holi==True]
# 1900개행이 아닌 1800개 --> 20171003이 개천절과 추석이 겹쳐있기 때문에 중복!

In [ ]:
# 주말이거나 공휴일이면 holi가 True가 되도록 변경
sub2["holi"]=np.where((sub2["요일"].isin(["토","일"]))|(sub2["holi"]==True),True,False)

In [ ]:
sub2[sub2.holi==True].head(60)

```
csv파일 생성 완료라 markdown으로 바꿔놓음
sub2.to_csv("sub2.csv",index=False,encoding="euc-kr")
```

## 신설동라인같은 역 제거 필요!!!

```





```

### 기상 데이터 전처리